In [2]:
import os
import asyncio
import httpx
import pandas as pd

# MADE BY VICTOR DUARTE. VDUART10
base_url = "https://www.gsar.ford.com/onlinereports/be/getVehInfoFull"

In [11]:
URL = "https://corp.sts.ford.com/adfs/oauth2/authorize"
PARAMS = {
    "client_id": "urn:gsar:clientid:web:prod",
    "response_type": "token id_token",
    "redirect_uri": "https://www.gsar.ford.com/oauth-callback/",
    "resource": "urn:gsar:resource:web:prod",
    "client-request-id": "7e0748c5-e9d3-4aeb-cd02-0080020000a9",
    "RedirectToIdentityProvider": "AD AUTHORITY",
}
DATA = {
    "UserName": str(os.getenv("GSAR_USERNAME")),
    "Password": str(os.getenv("GSAR_PASSWORD")),
    "AuthMethod": "FormsAuthentication",
}
response = httpx.post(URL, params=PARAMS, json=DATA, timeout=50)
response.status_code

200

In [4]:
def create_client() -> httpx.AsyncClient:
    """
    Creates a common client for future http requests

    Returns:
        httpx.Client: client with ford proxies
    """
    limits = httpx.Limits(max_connections=8)
    ford_proxy = httpx.Proxy(str(os.getenv("FORD_PROXY")))
    timeout_config = httpx.Timeout(10.0, connect=5.0, pool=4.0)
    proxy_mounts = {
        "http://": httpx.AsyncHTTPTransport(proxy=ford_proxy, limits=limits, retries=3),
        "https://": httpx.AsyncHTTPTransport(
            proxy=ford_proxy, limits=limits, retries=3
        ),
    }
    return httpx.AsyncClient(
        timeout=timeout_config,
        mounts=proxy_mounts,
        verify=False,
    )

In [5]:
async def get_info_by_vin(vin: str, client: httpx.AsyncClient, token: str):
    keys = ["prodDate", "wersVl", "awsVl", "globVl", "plant", "origWarantDate"]
    retrived = dict.fromkeys(keys, "")

    if vin and vin[-1] != "*" and len(vin) == 17:
        response = await client.get(
            base_url,
            params={"vin": vin},
            headers={
                "Authorization": f"Bearer {token}",
            },
        )
        data = dict(response.json())
        for key in keys:
            if key in data:

                retrived[key] = str(data.get(key))

    return list(retrived.values())


async def collect_vins_info(data: pd.DataFrame) -> pd.DataFrame:
    token = "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6ImFSZ2hZU01kbXI2RFZpMTdWVVJtLUJlUENuayJ9.eyJhdWQiOiJ1cm46Z3NhcjpyZXNvdXJjZTp3ZWI6cHJvZCIsImlzcyI6Imh0dHBzOi8vY29ycC5zdHMuZm9yZC5jb20vYWRmcy9zZXJ2aWNlcy90cnVzdCIsImlhdCI6MTcwOTczMTI4NCwiZXhwIjoxNzA5NzYwMDg0LCJDb21tb25OYW1lIjoiVkRVQVJUMTAiLCJzdWIiOiJWRFVBUlQxMCIsInVpZCI6InZkdWFydDEwIiwiZm9yZEJ1c2luZXNzVW5pdENvZGUiOiJGU0FNUiIsImdpdmVuTmFtZSI6IlZpY3RvciIsInNuIjoiRHVhcnRlIiwiaW5pdGlhbHMiOiJWLiIsIm1haWwiOiJ2ZHVhcnQxMEBmb3JkLmNvbSIsImVtcGxveWVlVHlwZSI6Ik0iLCJzdCI6IkJBIiwiYyI6IkJSQSIsImZvcmRDb21wYW55TmFtZSI6IklOU1QgRVVWQUxETyBMT0RJIE4gUkVHSU9OQUwgQkFISUEiLCJmb3JkRGVwdENvZGUiOiIwNjY0Nzg0MDAwIiwiZm9yZERpc3BsYXlOYW1lIjoiRHVhcnRlLCBWaWN0b3IgKFYuKSIsImZvcmREaXZBYmJyIjoiUFJEIiwiZm9yZERpdmlzaW9uIjoiUEQgT3BlcmF0aW9ucyBhbmQgUXVhbGl0eSIsImZvcmRDb21wYW55Q29kZSI6IjAwMDE1ODM4IiwiZm9yZE1hbmFnZXJDZHNpZCI6Im1tYWdyaTEiLCJmb3JkTVJSb2xlIjoiTiIsImZvcmRTaXRlQ29kZSI6IjY1MzYiLCJmb3JkVXNlclR5cGUiOiJFbXBsb3llZSIsImFwcHR5cGUiOiJQdWJsaWMiLCJhcHBpZCI6InVybjpnc2FyOmNsaWVudGlkOndlYjpwcm9kIiwiYXV0aG1ldGhvZCI6InVybjpvYXNpczpuYW1lczp0YzpTQU1MOjIuMDphYzpjbGFzc2VzOlBhc3N3b3JkUHJvdGVjdGVkVHJhbnNwb3J0IiwiYXV0aF90aW1lIjoiMjAyNC0wMy0wNlQxMzoyNjoyMy45NzJaIiwidmVyIjoiMS4wIn0.kh7uPNLPrHmPCQ1xEE5tai2qyOCNwiPdmzOYLZUFu0TzgauCWeRKKRFfmcwFErmFFe__NQyu5PrzViTHrZg9grr1KdLV7QxVSXjtLgkJOR0cNmII_PB_vi4qehUbeGHKiCaZW_zqs-V2eNDKAuLVeYdDeMIUw7bweJmL1DL3cpitETMKU3IfZDCf17Hnug_RxsXtwAh5uTtk4AdzQ7xlEAVYkdwAX-kVCcahXhJxIZ2MzXpreVxfDBC8Ej-_2eoXu9l8EFkUr8ykr04WpWIbqmIHO4VKau4nIltZPpqE99iYh24G-tubMuzJQ45hEBlGRozpxO-QhzscrTmdD1G3GA"
    async with create_client() as client:
        data[
            [
                "PROD_DATE",
                "VEHICLE_LINE_WERS",
                "VEHICLE_LINE_GSAR",
                "VEHICLE_LINE_GLOBAL",
                "ASSEMBLY_PLANT",
                "WARRANTY_START_DATE",
            ]
        ] = await asyncio.gather(
            *(get_info_by_vin(vin, client, token) for vin in data["VIN"])
        )
    data["PROD_DATE"] = (
        pd.to_datetime(data["PROD_DATE"], format="%d-%b-%Y", errors="coerce")
        .dt.strftime("%m/%d/%Y")
        .replace({"NaT": ""}, regex=True)
    )
    data["WARRANTY_START_DATE"] = (
        pd.to_datetime(data["WARRANTY_START_DATE"], format="%d-%b-%Y", errors="coerce")
        .dt.strftime("%m/%d/%Y")
        .replace({"NaT": ""}, regex=True)
    )
    return data

In [ ]:
vins_df = pd.read_excel(
    "../../../../data/external/NSCCV-000502-20240304.xlsx", sheet_name="VOQS"
)  # 29750
vins_df.dropna(inplace=True)
vins_df = vins_df[
    ~vins_df["VIN"].str.endswith("*")
    & (vins_df["VIN"] != "")
    & (vins_df["VIN"] != "N/A")
]
vins_df = await collect_vins_info(vins_df)
vins_df.head()